In [ ]:
import tensorflow as tf
from python_speech_features import *
import os
import keyboard as kb
import pyaudio
import csv
import numpy as np

SUB_FOLDER = "南无阿弥陀佛"
ID = 0
CSV_FILE_DIR = r"D:/语音识别/念佛计数/语言模型CSV/{0}".format(SUB_FOLDER)

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 60
pad = pyaudio.PyAudio()
stream = pad.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

model_epd = tf.keras.models.load_model(r"D:\语音识别\念佛计数\h5\PFB_EndPointDetect.h5")

def REC_preprocessing(soundBytes):
    #soundBytes = (soundBytes - soundBytes.mean())/soundBytes.std()
    mfcc_feat0 =  mfcc(soundBytes)
    mfcc_feat1 = delta(mfcc_feat0, 1)
    mfcc_feat2 = delta(mfcc_feat0, 2)
    feature = np.hstack((mfcc_feat0, mfcc_feat1, mfcc_feat2))
    feature = feature.reshape(-1,39)
    return feature

def EPD_preprocessing(soundBytes):
    mean = soundBytes.mean()
    std = soundBytes.std()
    soundBytes = (soundBytes - mean)/std
    return soundBytes

def EPD_predict(data, seq_length):
    soundBytes = np.fromstring(np.array(data), dtype=np.int16)
    pfe = EPD_preprocessing(soundBytes).reshape(1,seq_length,1)
    epd = int(np.round(model_epd.predict(pfe)))
    return epd, soundBytes

print("开始录音")
features = []
labels = []
RECORD_LENGHT = int(RATE / CHUNK * RECORD_SECONDS)
for t in range(0, RECORD_LENGHT):
    print("\r" + "recording:{0}/{1}".format(t, RECORD_LENGHT), end="", flush=True)
    data = stream.read(CHUNK)
    epd, soundBytes = EPD_predict(data, CHUNK)
    if epd == 1:
        feature = REC_preprocessing(soundBytes)
        features.append(feature)
        if kb.is_pressed("ctrl") :
            labels.append(1)
        else:
            labels.append(0)
print("结束录音")
print("features:", np.shape(features))
print("labels:", np.shape(labels))


def SaveCSV(path, x, y):
    sx = np.array(x.copy())
    sx = sx.reshape(sx.shape[0], sx.shape[1] * sx.shape[2])
    sy = np.array(y.copy())
    sy = sy.reshape(sy.shape[0], 1)
    s = np.hstack((sx, sy))
    print("save data shape:", s.shape)
    with open(path, 'w', newline='') as csv_file:
        csv_w = csv.writer(csv_file)
        for l in s:
            csv_w.writerow(l)
    print("save csv successed!:{0} ".format(path))


filelist = os.listdir(CSV_FILE_DIR)
CLIP_NAME = len(filelist)
CLIP_NAME_STR = "{0}/{1}_{2}.csv".format(CSV_FILE_DIR, "%03d" % ID, "%03d" % CLIP_NAME)
SaveCSV(CLIP_NAME_STR, features, labels)